In [1]:
import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
#Se leen los datos preparados en el EDA
df = pd.read_csv("data\df_modelo_recomendacion.csv")

In [3]:
#Se genera un datafram con el user_id como indice, itenm_name como columna, y raiting como valor.
df_pivot = df.pivot_table(index=["user_id"], columns=["item_name"], values="raiting")

In [4]:
df_pivot

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,123 Slaughter Me Street,140,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theBlu,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ace--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ionex--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2SV-vuLB-Kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Azsael-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zv_odd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zvanik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zwanzigdrei,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se normaliza el dataframe,resta la media de calificaciones y luego se divide por la diferencia entre el valor maximo y minimo.

A los usuarios con una sola calificacion o que califican a todos los jeugos de la misma manera seran eliminados durante este proceso ya que no dan informacion util.

In [5]:
df_pivot = df_pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)) ,axis=1)
#se borral las columnas que solo tienen nulos o no tienen raiting,los vacios se rellenan con 0
df_pivot.fillna(0,inplace =True)
df_pivot =df_pivot.T
df_pivot=df_pivot.loc[:, (df_pivot !=0).any(axis=0)]

In [6]:
df_pivot

user_id,--000--,-Beave-,-GM-Dragon,-I_AM_EPIC-,-SEVEN-,-Thyme-,-kainey9777,00000000000000000001227,00690069006900,01189958889189157253,...,ziqan,zjaerya135,zombi_anon,zomgCoBfAce,zoom-the-flash,zoozles,zourock,zsharoarkbr,zvanik,zzoptimuszz
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001 Spikes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
theBlu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A los datos normalizados se le da un formato de matriz dispersa para reducir el uso de memoria en el manejo de datos

In [7]:
pivot_sparse = sp.sparse.csr_matrix(df_pivot.values)

Se crean dos matrices de similutd usando la similitud de coseno para medir la similitud entre los juegos y entre los usuarios.
Ña similud de conseno se calcula utilizando el coseno del ángulo entre los vectores que representan esos datos o elementos.

In [8]:
item_sim = cosine_similarity(pivot_sparse)
user_sim = cosine_similarity(pivot_sparse.T)

In [9]:
#Se crean datafram con las matrices para organizarse de manera comprensible.
df_item_sim = pd.DataFrame(item_sim, index = df_pivot.index, columns = df_pivot.index)
df_user_sim = pd.DataFrame(user_sim, index = df_pivot.columns, columns = df_pivot.columns)

Funcion de recomendacion item a item

Conociendo la relacion entre juegos, se puede pronoer una funcion de recomendacionde 5 juegos segun un juego dado, teniendo en cuenta la similitud de coseno.

In [10]:
def game(game):
    '''
    Muestra la lista de juegos similares

    parameters:
        game(str) El nombre del juego que se desea buscar similares

    return
        list: Lista de 5 jeugos similares
    
    '''
    count = 1
    print("juegos similares{}\n".format(game))
    for item in df_item_sim.sort_values(by = game, ascending = False).index[1:6]:
        print(f"Nuumero{count}: {item}")
        count+=1
    

In [11]:
game("1001 Spikes")

juegos similares1001 Spikes

Nuumero1: Super Meat Boy
Nuumero2: ! That Bastard Is Trying To Steal Our Gold !
Nuumero3: Revelations 2012
Nuumero4: Resident Evil Revelations 2 / Biohazard Revelations 2
Nuumero5: Resident Evil™: Operation Raccoon City


Funcion de recomendacion user a item

Una funcion de recomendacion que dando un usuario nos devuelve una lista de 5 juegos .

In [12]:
#primero se nesesita una funcion que busque usuarios similares

def user_similarity(user):
    '''
    Devuelve una lista con los usuarios similares.

    parameters:
        user(str):el id del usuario

    return:
        Lista: lista con los usuarios similares.
    '''

     #se revisa que el usuario este en la lista
    if user not in df_pivot.columns:
        return("usuario no encontrado")
    print(f"usuarios Similares a:{user}")
    sim_values = df_user_sim.sort_values(by=user, ascending = False).loc[:,user].tolist()[1:6]
    sim_user = df_user_sim.sort_values(by=user, ascending=False).index[1:11]
    zipped = zip(sim_user,sim_values,)


    for user,sim in zipped:
        print(f"Usuario{user}, usuarios similar: {round(sim,2)}")



In [13]:
user_similarity('zvanik')

usuarios Similares a:zvanik


Usuario76561197966979423, usuarios similar: 0.61
Usuario123123718417671290, usuarios similar: 0.61
Usuarioplskeelkan, usuarios similar: 0.58
Usuariotsunamitad, usuarios similar: 0.58
Usuario76561197987617385, usuarios similar: 0.58


In [14]:
def game_user(user):
    '''
    Genera una lista de 5 juegos mas recomendados según un usuario.
    parameters:
        user (str): El nombre del usuario para las recomendaciones.
    return:
        list: Lista de 5 juegos recomendados
    '''

    # se revisa que el usuario esté en la lista
    if user not in df_pivot.columns:
        return "usuario no encontrado"
    
    sim_user = df_user_sim.sort_values(by=user, ascending=False).index[1:11]
    games = []  # lista de juegos mejor calificados por usuarios similares
    count = {}  # diccionario para contar cuántas veces se recomendó cada juego.

    for i in sim_user:
        max_rating = df_pivot.loc[:, i].max()
        games.extend(df_pivot[df_pivot.loc[:, i] == max_rating].index.tolist())

    for game in games:
        if game in count:
            count[game] += 1
        else:
            count[game] = 1
    
    sorted_list = sorted(count.items(), key=operator.itemgetter(1), reverse=True)

    return sorted_list[:5]


In [15]:
game_user("zvanik")

[('Call of Duty: World at War', 5),
 ('Far Cry 4', 1),
 ('Fistful of Frags', 1),
 ('H1Z1: Just Survive', 1),
 ('BattleBlock Theater', 1)]

In [16]:
# se guardan las matrcioes que se nesesitan para usar los modelos.

pq.write_table(pa.Table.from_pandas(df_pivot),"data\df_pivot.parquet")
pq.write_table(pa.Table.from_pandas(df_item_sim),"data\df_item_sim.parquet")
pq.write_table(pa.Table.from_pandas(df_user_sim),"data\df_user_sim.parquet")

In [ ]:
file_path_gzip = "data\df_item_sim_gzip.parquet"
df_item_sim.to_parquet(file_path_gzip, engine="pyarrow", compression="gzip")